In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch

from transformers import BertTokenizer
from typing import *

from MEOW_Models.ST_model import*
from MEOW_Models.MT_models import MEOW_MTM
from MEOW_Models.Kernel_model import BertWithoutEmbedding
from MEOW_Utils import Data_utils, MT_utils

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 4
EPOCH_NUM = 10

INPUT_FILE_PATH_SQuAD = r'C:\Users\Administrator\codeblocks_workspace\MEOW\Data_file\SQuAD.csv'
INPUT_FILE_PATH_CoLA = r'C:\Users\Administrator\codeblocks_workspace\MEOW\Data_file\CoLA_train.csv'
INPUT_FILE_PATH_MNLI = r'C:\Users\Administrator\codeblocks_workspace\MEOW\Data_file\MNLI_train.csv'
INPUT_FILE_PATH_RTE = r'C:\Users\Administrator\codeblocks_workspace\MEOW\Data_file\RTE_train.csv'
INPUT_FILE_PATH_Sentiment = r'C:\Users\Administrator\codeblocks_workspace\MEOW\Data_file\Sentiment_train.csv'

SQuAD_DATASIZE = 500
CoLA_DATASIZE = 200
Sentiment_DATASIZE = 350
MNLI_DATASIZE = 300
RTE_DATASIZE = 200

CoLA_num_labels = 2
Sentiment_num_labels = 5
MNLI_num_labels = 3
RTE_num_labels = 2

# SQuAD_DATASIZE max value is 5000
# CoLA_DATASIZE max value is 5056
# Sentiment_DATASIZE max value is 27405
# MNLI_DATASIZE max value is 13191
# RTE_DATASIZE max value is 2482

PRETRAINED_MODULE_NAME = 'bert-base-uncased'

In [4]:
#處理好 dataframe
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODULE_NAME)

df_train_CoLA = Data_utils.get_CoLA_df(INPUT_FILE_PATH_CoLA, CoLA_DATASIZE)

df_train_Sentiment = Data_utils.get_Sentiment_df(INPUT_FILE_PATH_Sentiment, Sentiment_DATASIZE)

df_train_MNLI = Data_utils.get_MNLI_df(INPUT_FILE_PATH_MNLI, tokenizer, MNLI_DATASIZE)

df_train_RTE = Data_utils.get_RTE_df(INPUT_FILE_PATH_RTE, tokenizer, RTE_DATASIZE)

df_train_SQuAD = Data_utils.get_SQuAD_df(INPUT_FILE_PATH_SQuAD, tokenizer, SQuAD_DATASIZE)

c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\Data_utils.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['TKstart'][i], df_train['TKend'][i] = count_the_TKbeg_and_TKend(df_train.iloc[i]['context'], df_train.iloc[i]['answer_start'], df_train.iloc[i]['text'], tokenizer)


In [3]:
CoLA_dataset = Data_utils.get_CoLA_dataset(df_train_CoLA, tokenizer, CoLA_num_labels)

Sentiment_dataset = Data_utils.get_Sentiment_dataset(df_train_Sentiment, tokenizer, Sentiment_num_labels)

MNLI_dataset = Data_utils.get_MNLI_dataset(df_train_MNLI, tokenizer, MNLI_num_labels)

RTE_dataset = Data_utils.get_RTE_dataset(df_train_RTE, tokenizer, RTE_num_labels)

SQuAD_dataset = Data_utils.get_SQuAD_dataset(df_train_SQuAD, tokenizer)

In [4]:
CoLA_dataloader = Data_utils.get_CoLA_dataloader(CoLA_dataset, batch_size=BATCH_SIZE)

Sentiment_dataloader = Data_utils.get_Sentiment_dataloader(Sentiment_dataset, batch_size=BATCH_SIZE)

MNLI_dataloader = Data_utils.get_MNLI_dataloader(MNLI_dataset, batch_size=BATCH_SIZE)

RTE_dataloader = Data_utils.get_RTE_dataloader(RTE_dataset, batch_size=BATCH_SIZE)

SQuAD_dataloader = Data_utils.get_SQuAD_dataloader(SQuAD_dataset, batch_size=BATCH_SIZE)

In [7]:
# check if the dataloader can run correctly 
for i in CoLA_dataloader:
    0
for i in Sentiment_dataloader:
    0
for i in MNLI_dataloader:
    0
for i in RTE_dataloader:
    0
for i in SQuAD_dataloader:
    0

In [5]:
#給每個任務embedding層

bert_kernel_model = BertWithoutEmbedding.from_pretrained(PRETRAINED_MODULE_NAME)
GETEMBEDDING_helper =  MT_utils.get_bert_element(bertmodel=bert_kernel_model)

CoLA_embedding_layer = GETEMBEDDING_helper.get_copy_embeddings_layer()

Sentiment_embedding_layer = GETEMBEDDING_helper.get_copy_embeddings_layer()

MNLI_embedding_layer = GETEMBEDDING_helper.get_copy_embeddings_layer()

RTE_embedding_layer = GETEMBEDDING_helper.get_copy_embeddings_layer()

SQuAD_embedding_layer = GETEMBEDDING_helper.get_copy_embeddings_layer()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithoutEmbedding: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertWithoutEmbedding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWithoutEmbedding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
MEOW_model = MEOW_MTM(bert_kernel_model, 
                    CoLA_embedding_layer, CoLA_num_labels,
                    Sentiment_embedding_layer, Sentiment_num_labels, 
                    MNLI_embedding_layer, MNLI_num_labels, 
                    RTE_embedding_layer, RTE_num_labels,
                    SQuAD_embedding_layer,
                    DEVICE)

H_CoLA = {
    "train_loss": [],
    "train_acc": [],
    "val_loss":[],
    "val_acc": []
    }

H_Sentiment = {
    "train_loss": [],
    "train_acc": [],
    "val_loss":[],
    "val_acc": []
    }

H_MNLI = {
    "train_loss": [],
    "train_acc": [],
    "val_loss":[],
    "val_acc": []
    }

H_RTE = {
    "train_loss": [],
    "train_acc": [],
    "val_loss":[],
    "val_acc": []
    }

H_SQuAD = {
    "train_loss": [],
    "train_acc": [],
    "val_loss":[],
    "val_acc": []
    }

In [6]:
a_epoc_data_num = min([len(CoLA_dataset), len(Sentiment_dataset), len(MNLI_dataset), len(RTE_dataset), len(SQuAD_dataset)])
round = min([len(CoLA_dataloader), len(Sentiment_dataloader), len(MNLI_dataloader), len(RTE_dataloader), len(SQuAD_dataloader)])
print(f'Per epoc round\'s num is {round}')

NameError: name 'CoLA_dataloader' is not defined

In [ ]:
# 訓練

for epoch in range(EPOCH_NUM):
    print("the {:d} iter :".format(epoch+1))

    MEOW_model.train()

    CoLA_training_loss = 0.0
    Sentiment_training_loss = 0.0
    MNLI_training_loss = 0.0
    RTE_training_loss = 0.0
    SQuAD_training_loss = 0.0
    MT_training_loss = 0.0

    CoLA_correct_num = 0.0
    Sentiment_correct_num = 0.0
    MNLI_correct_num = 0.0
    RTE_correct_num = 0.0

    CoLA_iter = iter(CoLA_dataloader)
    Sentiment_iter = iter(Sentiment_dataloader)
    MNLI_iter = iter(MNLI_dataloader)
    RTE_iter = iter(RTE_dataloader)
    SQuAD_iter = iter(SQuAD_dataloader)

    # training_correct = 0.0

    for i in range(round):
        
        # CoLA first
        input_ids, mask, token, label = next(CoLA_iter)

        input_ids = input_ids.to(DEVICE)
        mask = mask.to(DEVICE)
        token = token.to(DEVICE)
        label = label.to(DEVICE)
        
        loss, prob = MEOW_model.CoLA_forward(input_ids, mask, token, label)
        #print("the CoLA loss is : ", loss.item())
        MEOW_model.optimize_CoLA(loss)
        
        CoLA_training_loss += loss.item()
        MT_training_loss += loss.item()
        predict = torch.argmax(prob, dim=1)
        label = torch.argmax(label, dim=1)
        CoLA_correct_num += (predict == label).type(torch.int).sum()

        # Sentiment
        input_ids, mask, token, label = next(Sentiment_iter)

        input_ids = input_ids.to(DEVICE)
        mask = mask.to(DEVICE)
        token = token.to(DEVICE)
        label = label.to(DEVICE)
        
        loss, prob = MEOW_model.Sentiment_forward(input_ids, mask, token, label)
        #print("the Sentiment loss is : ", loss.item())
        MEOW_model.optimize_Sentiment(loss)
        
        Sentiment_training_loss += loss.item()
        MT_training_loss += loss.item()
        predict = torch.argmax(prob, dim=1)
        label = torch.argmax(label, dim=1)
        Sentiment_correct_num += (predict == label).type(torch.int).sum()

        # MNLI
        input_ids, mask, token, label, SEPind  = next(MNLI_iter)

        input_ids = input_ids.to(DEVICE)
        mask = mask.to(DEVICE)
        token = token.to(DEVICE)
        label = label.to(DEVICE)
        
        loss, prob = MEOW_model.MNLI_forward(input_ids, mask, token, label, SEPind)
        #print("the MNLI loss is : ", loss.item())
        MEOW_model.optimize_MNLI(loss)
        
        MNLI_training_loss += loss.item()
        MT_training_loss += loss.item()
        predict = torch.argmax(prob, dim=1)
        label = torch.argmax(label, dim=1)
        MNLI_correct_num += (predict == label).type(torch.int).sum()

        # RTE
        input_ids, mask, token, label, SEPind  = next(RTE_iter)

        input_ids = input_ids.to(DEVICE)
        mask = mask.to(DEVICE)
        token = token.to(DEVICE)
        label = label.to(DEVICE)
        
        loss, prob = MEOW_model.RTE_forward(input_ids, mask, token, label, SEPind)
        #print("the RTE loss is : ", loss.item())
        MEOW_model.optimize_RTE(loss)
        
        RTE_training_loss += loss.item()
        MT_training_loss += loss.item()
        predict = torch.argmax(prob, dim=1)
        label = torch.argmax(label, dim=1)
        RTE_correct_num += (predict == label).type(torch.int).sum()

        # SQuAD
        input_ids, mask, token, SEPind, Start_pos, End_pos = next(SQuAD_iter)

        input_ids = input_ids.to(DEVICE)
        mask = mask.to(DEVICE)
        token = token.to(DEVICE)
        label = label.to(DEVICE)
        
        loss = MEOW_model.SQuAD_forward(input_ids, mask, token, SEPind, Start_pos, End_pos)
        # print("the SQuAD loss is : ", loss.item())
        MEOW_model.optimize_SQuAD(loss)
        
        SQuAD_training_loss += loss.item()
        MT_training_loss += loss.item()


    CoLA_avg_loss = CoLA_training_loss / round
    Sentiment_avg_loss = Sentiment_training_loss / round
    MNLI_avg_loss = MNLI_training_loss / round
    RTE_avg_loss = RTE_training_loss / round
    SQuAD_avg_loss = SQuAD_training_loss / round
    
    CoLA_accuracy = CoLA_correct_num / a_epoc_data_num
    Sentiment_accuracy = Sentiment_correct_num / a_epoc_data_num
    MNLI_accuracy = MNLI_correct_num / a_epoc_data_num
    RTE_accuracy = RTE_correct_num / a_epoc_data_num
    
    H_CoLA['train_loss'].append(CoLA_avg_loss)
    H_Sentiment['train_loss'].append(Sentiment_avg_loss)
    H_MNLI['train_loss'].append(MNLI_avg_loss)
    H_RTE['train_loss'].append(RTE_avg_loss)
    H_SQuAD['train_loss'].append(SQuAD_avg_loss)
    
    H_CoLA['train_acc'].append(CoLA_accuracy)
    H_Sentiment['train_acc'].append(Sentiment_accuracy)
    H_MNLI['train_acc'].append(MNLI_accuracy)
    H_RTE['train_acc'].append(RTE_accuracy)

    # H['train_acc'].append(avg_correct)
    print("CoLA train loss: {:.6f}, CoLA accuracy: {:.4f}".format(CoLA_avg_loss, CoLA_accuracy))
    print("Sentiment train loss: {:.6f}, Sentiment accuracy: {:.4f}".format(Sentiment_avg_loss, Sentiment_accuracy))
    print("MNLI train loss: {:.6f}, MNLI accuracy: {:.4f}".format(MNLI_avg_loss, MNLI_accuracy))
    print("RTE train loss: {:.6f}, RTE accuracy: {:.4f}".format(RTE_avg_loss, RTE_accuracy))
    print("SQuAD train loss: {:.6f}".format(SQuAD_avg_loss))

In [ ]:
MT_utils.plot_diagram(H=H_CoLA, epoch_num=EPOCH_NUM, has_accuracy=True)